# OpenVINO™를 사용하여 YOLOv8 실시간 개체 감지를 변환하고 최적화합니다.

실시간 객체 감지는 종종 컴퓨터 비전 시스템의 핵심 구성 요소로 사용됩니다.
실시간 객체 감지 모델을 사용하는 애플리케이션에는 비디오 분석, 로봇 공학, 자율 주행 차량, 다중 객체 추적 및 객체 계산, 의료 이미지 분석 등이 포함됩니다.


이 튜토리얼에서는 OpenVINO를 사용하여 PyTorch YOLOv8을 실행하고 최적화하는 방법에 대한 단계별 지침을 보여줍니다. 객체 감지 시나리오에 필요한 단계를 고려합니다.

튜토리얼은 다음 단계로 구성됩니다.
- PyTorch 모델을 준비합니다.
- 데이터 세트를 다운로드하고 준비합니다.
- 원본 모델을 검증합니다.
- PyTorch 모델을 OpenVINO IR로 변환합니다.
- 변환된 모델을 검증합니다.
- 최적화 파이프라인을 준비하고 실행합니다.
- FP32와 양자화 모델의 성능을 비교합니다.
- FP32와 양자화 모델의 정확도를 비교합니다.
- OpenVINO API를 통한 기타 최적화 가능성
- 라이브 데모


#### 내용의 테이블:
- [PyTorch 모델 가져오기](#Get-PyTorch-model-Uparrow)
     - [선행조건](#선행조건-Uparrow)
- [모델 인스턴스화](#Instantiate-model-Uparrow)
     - [모델을 OpenVINO IR로 변환](#Convert-model-to-OpenVINO-IR-Uparrow)
     - [모델 추론 검증](#Verify-model-inference-Uparrow)
     - [전처리](#전처리-Uparrow)
     - [후처리](#후처리-Uparrow)
     - [추론 장치 선택](#Select-inference-device-Uparrow)
     - [단일 이미지 테스트](#Test-on-single-image-Uparrow)
- [데이터세트의 모델 정확도 확인](#Check-model-accuracy-on-the-dataset-Uparrow)
     - [검증 데이터 세트 다운로드](#Download-the-validation-dataset-Uparrow)
     - [검증 함수 정의](#Define-validation-function-Uparrow)
     - [검사기 도우미 구성 및 DataLoader 생성](#Configure-Validator-helper-and-create-DataLoader-Uparrow)
- [NNCF 사후 훈련 양자화 API를 사용하여 모델 최적화](#Optimize-model-using-NNCF-Post-training-Quantization-API-Uparrow)
     - [양자화 모델 추론 검증](#Validate-Quantized-model-inference-Uparrow)
- [원본 모델과 양자화 모델 비교](#Compare-the-Original-and-Quantized-Models-Uparrow)
     - [성능 객체 감지 모델 비교](#Compare-performance-object-Detection-models-Uparrow)
     - [양자화 모델 정확도 검증](#Validate-Quantized-model-accuracy-Uparrow)
- [다음 단계](#다음-단계-위쪽 화살표)
     - [비동기 추론 파이프라인](#Async-inference-pipeline-Uparrow)
     - [모델로의 통합 전처리](#Integration-preprocessing-to-model-Uparrow)
         - [PrePostProcessing API 초기화](#Initialize-PrePostProcessing-API-Uparrow)
         - [입력 데이터 형식 정의](#Define-input-data-format-Uparrow)
         - [전처리 단계 설명](#Describe-preprocessing-steps-Uparrow)
         - [모델에 단계 통합](#Integrating-Steps-into-a-Model-Uparrow)
- [라이브 데모](#Live-demo-Uparrow)
     - [라이브 객체 감지 실행](#Run-Live-Object-Detection-Uparrow)

## PyTorch 모델 가져오기 [$\Uparrow$](#목차:)

일반적으로 PyTorch 모델은 모델이 포함된 상태 사전에 의해 초기화된 [`torch.nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) 클래스의 인스턴스를 나타냅니다. 무게.
우리는 이 [repo](https://github.com/ultralytics/ultralytics)에서 사용할 수 있는 COCO 데이터세트에서 사전 훈련된 YOLOv8 나노 모델('yolov8n'이라고도 함)을 사용합니다. 비슷한 단계가 다른 YOLOv8 모델에도 적용 가능합니다.
사전 훈련된 모델을 얻기 위한 일반적인 단계:
1. 모델 클래스의 인스턴스를 생성합니다.
2. 사전 훈련된 모델 가중치가 포함된 체크포인트 상태 사전을 로드합니다.
3. 일부 작업을 추론 모드로 전환하기 위해 모델을 평가로 전환합니다.

이 경우 모델 작성자는 YOLOv8 모델을 ONNX로 변환한 다음 OpenVINO IR로 변환할 수 있는 API를 제공합니다. 따라서 이러한 단계를 수동으로 수행할 필요가 없습니다.

#### 전제조건 [$\Uparrow$](#목차:)

필요한 패키지를 설치합니다.

In [ ]:
%pip install -q "openvino>=2023.1.0" "nncf>=2.5.0"
%pip install -q "ultralytics==8.0.43" onnx

필수 유틸리티 기능을 가져옵니다.
아래쪽 셀은 GitHub에서 'notebook_utils' Python 모듈을 다운로드합니다.

In [ ]:
from pathlib import Path

# Fetch the notebook utils script from the openvino_notebooks repo
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)

from notebook_utils import download_file, VideoPlayer

결과 그리기를 위한 유틸리티 함수 정의

In [ ]:
from typing import Tuple, Dict
import cv2
import numpy as np
from ultralytics.yolo.utils.plotting import colors


def plot_one_box(box:np.ndarray, img:np.ndarray,
                 color:Tuple[int, int, int] = None,
                 label:str = None, line_thickness:int = 5):
    """
    Helper function for drawing single bounding box on image
    Parameters:
        x (np.ndarray): bounding box coordinates in format [x1, y1, x2, y2]
        img (no.ndarray): input image
        color (Tuple[int, int, int], *optional*, None): color in BGR format for drawing box, if not specified will be selected randomly
        label (str, *optonal*, None): box label string, if not provided will not be provided as drowing result
        line_thickness (int, *optional*, 5): thickness for box drawing lines
    """
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

    return img


def draw_results(results:Dict, source_image:np.ndarray, label_map:Dict):
    """
    Helper function for drawing bounding boxes on image
    Parameters:
        image_res (np.ndarray): detection predictions in format [x1, y1, x2, y2, score, label_id]
        source_image (np.ndarray): input image for drawing
        label_map; (Dict[int, str]): label_id to class name mapping
    Returns:
        Image with boxes
    """
    boxes = results["det"]
    for idx, (*xyxy, conf, lbl) in enumerate(boxes):
        label = f'{label_map[int(lbl)]} {conf:.2f}'
        source_image = plot_one_box(xyxy, source_image, label=label, color=colors(int(lbl)), line_thickness=1)
    return source_image

In [ ]:
# Download a test sample
IMAGE_PATH = Path('./data/coco_bike.jpg')
download_file(
    url='https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/image/coco_bike.jpg',
    filename=IMAGE_PATH.name,
    directory=IMAGE_PATH.parent
)

## 모델 인스턴스화 [$\Uparrow$](#목차:)

원본 저장소에는 다양한 작업을 대상으로 하는 [여러 모델](https://docs.ultralytics.com/tasks/Detect/)이 있습니다. 모델을 로드하려면 모델 체크포인트에 대한 경로를 지정해야 합니다. 모델 허브에서 사용할 수 있는 일부 로컬 경로 또는 이름일 수 있습니다(이 경우 모델 체크포인트는 자동으로 다운로드됩니다).

예측을 수행하면 모델은 입력 이미지에 대한 경로를 수락하고 Results 클래스 개체가 포함된 목록을 반환합니다. 결과에는 객체 감지 모델에 대한 상자가 포함됩니다. 또한 결과를 처리하기 위한 유틸리티(예: 그리기를 위한 `plot()` 메서드)도 포함되어 있습니다.

다음의 예를 고려해 보겠습니다.

In [ ]:
models_dir = Path('./models')
models_dir.mkdir(exist_ok=True)

In [ ]:
from PIL import Image
from ultralytics import YOLO

DET_MODEL_NAME = "yolov8n"

det_model = YOLO(models_dir / f'{DET_MODEL_NAME}.pt')
label_map = det_model.model.names

res = det_model(IMAGE_PATH)
Image.fromarray(res[0].plot()[:, :, ::-1])

### 모델을 OpenVINO IR로 변환 [$\Uparrow$](#목차:)

YOLOv8은 OpenVINO IR을 포함한 다양한 형식으로 편리한 모델을 내보낼 수 있는 API를 제공합니다. `model.export`는 모델 변환을 담당합니다. 형식을 지정해야 하며 추가적으로 모델의 동적 형태를 보존할 수 있습니다.

In [ ]:
# object detection model
det_model_path = models_dir / f"{DET_MODEL_NAME}_openvino_model/{DET_MODEL_NAME}.xml"
if not det_model_path.exists():
    det_model.export(format="openvino", dynamic=True, half=False)

### 모델 추론 확인 [$\Uparrow$](#목차:)

모델 작업을 테스트하기 위해 'model.predict' 메서드와 유사한 추론 파이프라인을 생성합니다. 파이프라인은 전처리 단계, OpenVINO 모델 추론, 결과를 얻기 위한 결과 후처리로 구성됩니다.

### 전처리 [$\Uparrow$](#목차:)

모델 입력은 `N, C, H, W` 형식의 `[-1, 3, -1, -1]` 모양의 텐서입니다.
* `N` - 배치의 이미지 수(배치 크기)
* `C` - 이미지 채널
* `H` - 이미지 높이
* `W` - 이미지 너비

모델은 RGB 채널 형식의 이미지를 예상하고 [0, 1] 범위로 정규화됩니다. 모델은 입력 분할성을 32로 유지하면서 동적 입력 형태를 지원하지만 효율성을 높이기 위해 정적 형태(예: 640x640)를 사용하는 것이 좋습니다. 모델 크기 `letterbox`에 맞게 이미지 크기를 조정하려면 너비와 높이의 종횡비가 유지되는 크기 조정 접근 방식이 사용됩니다.

특정 모양을 유지하기 위해 전처리를 통해 자동으로 패딩이 활성화됩니다.

In [ ]:
from typing import Tuple
from ultralytics.yolo.utils import ops
import torch
import numpy as np


def letterbox(img: np.ndarray, new_shape:Tuple[int, int] = (640, 640), color:Tuple[int, int, int] = (114, 114, 114), auto:bool = False, scale_fill:bool = False, scaleup:bool = False, stride:int = 32):
    """
    Resize image and padding for detection. Takes image as input,
    resizes image to fit into new shape with saving original aspect ratio and pads it to meet stride-multiple constraints

    Parameters:
      img (np.ndarray): image for preprocessing
      new_shape (Tuple(int, int)): image size after preprocessing in format [height, width]
      color (Tuple(int, int, int)): color for filling padded area
      auto (bool): use dynamic input size, only padding for stride constrins applied
      scale_fill (bool): scale image to fill new_shape
      scaleup (bool): allow scale image if it is lower then desired input size, can affect model accuracy
      stride (int): input padding stride
    Returns:
      img (np.ndarray): image after preprocessing
      ratio (Tuple(float, float)): hight and width scaling ratio
      padding_size (Tuple(int, int)): height and width padding size


    """
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scale_fill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)


def preprocess_image(img0: np.ndarray):
    """
    Preprocess image according to YOLOv8 input requirements.
    Takes image in np.array format, resizes it to specific size using letterbox resize and changes data layout from HWC to CHW.

    Parameters:
      img0 (np.ndarray): image for preprocessing
    Returns:
      img (np.ndarray): image after preprocessing
    """
    # resize
    img = letterbox(img0)[0]

    # Convert HWC to CHW
    img = img.transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    return img


def image_to_tensor(image:np.ndarray):
    """
    Preprocess image according to YOLOv8 input requirements.
    Takes image in np.array format, resizes it to specific size using letterbox resize and changes data layout from HWC to CHW.

    Parameters:
      img (np.ndarray): image for preprocessing
    Returns:
      input_tensor (np.ndarray): input tensor in NCHW format with float32 values in [0, 1] range
    """
    input_tensor = image.astype(np.float32)  # uint8 to fp32
    input_tensor /= 255.0  # 0 - 255 to 0.0 - 1.0

    # add batch dimension
    if input_tensor.ndim == 3:
        input_tensor = np.expand_dims(input_tensor, 0)
    return input_tensor

### 후처리 [$\Uparrow$](#목차:)

모델 출력에는 탐지 상자 후보가 포함되어 있습니다. 이는 'B,84,N' 형식의 '[-1,84,-1]' 모양을 가진 텐서입니다. 여기서:

- `B` - 배치 크기
- `N` - 감지 상자 수

최종 예측을 얻으려면 최대가 아닌 억제 알고리즘을 적용하고 상자 좌표를 원래 이미지 크기로 다시 조정해야 합니다.

마지막으로 감지 상자에는 [`x`, `y`, `h`, `w`, `class_no_1`, ..., `class_no_80`] 형식이 있습니다.

- (`x`, `y`) - 상자 중심의 원시 좌표
- `h`, `w` - 상자의 원래 높이와 너비
- `class_no_1`, ..., `class_no_80` - 클래스에 대한 확률 분포입니다.

In [ ]:
def postprocess(
    pred_boxes:np.ndarray,
    input_hw:Tuple[int, int],
    orig_img:np.ndarray,
    min_conf_threshold:float = 0.25,
    nms_iou_threshold:float = 0.7,
    agnosting_nms:bool = False,
    max_detections:int = 300,
):
    """
    YOLOv8 model postprocessing function. Applied non maximum supression algorithm to detections and rescale boxes to original image size
    Parameters:
        pred_boxes (np.ndarray): model output prediction boxes
        input_hw (np.ndarray): preprocessed image
        orig_image (np.ndarray): image before preprocessing
        min_conf_threshold (float, *optional*, 0.25): minimal accepted confidence for object filtering
        nms_iou_threshold (float, *optional*, 0.45): minimal overlap score for removing objects duplicates in NMS
        agnostic_nms (bool, *optiona*, False): apply class agnostinc NMS approach or not
        max_detections (int, *optional*, 300):  maximum detections after NMS
    Returns:
       pred (List[Dict[str, np.ndarray]]): list of dictionary with det - detected boxes in format [x1, y1, x2, y2, score, label]
    """
    nms_kwargs = {"agnostic": agnosting_nms, "max_det":max_detections}
    preds = ops.non_max_suppression(
        torch.from_numpy(pred_boxes),
        min_conf_threshold,
        nms_iou_threshold,
        nc=80,
        **nms_kwargs
    )

    results = []
    for i, pred in enumerate(preds):
        shape = orig_img[i].shape if isinstance(orig_img, list) else orig_img.shape
        if not len(pred):
            results.append({"det": [], "segment": []})
            continue
        pred[:, :4] = ops.scale_boxes(input_hw, pred[:, :4], shape).round()
        results.append({"det": pred})

    return results

### 추론 장치 선택 [$\Uparrow$](#목차:)

OpenVINO를 사용하여 추론을 실행하려면 드롭다운 목록에서 장치를 선택하세요.

In [ ]:
import ipywidgets as widgets
import openvino as ov

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

### 단일 이미지에 대한 테스트 [$\Uparrow$](#목차:)

이제 전처리 및 후처리 단계를 정의했으면 객체 감지를 위한 모델 예측을 확인할 준비가 되었습니다.

In [ ]:
core = ov.Core()

det_ov_model = core.read_model(det_model_path)
if device.value != "CPU":
    det_ov_model.reshape({0: [1, 3, 640, 640]})
det_compiled_model = core.compile_model(det_ov_model, device.value)


def detect(image:np.ndarray, model:ov.Model):
    """
    OpenVINO YOLOv8 model inference function. Preprocess image, runs model inference and postprocess results using NMS.
    Parameters:
        image (np.ndarray): input image.
        model (Model): OpenVINO compiled model.
    Returns:
        detections (np.ndarray): detected boxes in format [x1, y1, x2, y2, score, label]
    """
    preprocessed_image = preprocess_image(image)
    input_tensor = image_to_tensor(preprocessed_image)
    result = model(input_tensor)
    boxes = result[model.output(0)]
    input_hw = input_tensor.shape[2:]
    detections = postprocess(pred_boxes=boxes, input_hw=input_hw, orig_img=image)
    return detections

input_image = np.array(Image.open(IMAGE_PATH))
detections = detect(input_image, det_compiled_model)[0]
image_with_boxes = draw_results(detections, input_image, label_map)

Image.fromarray(image_with_boxes)

## 데이터세트 [$\Uparrow$](#목차:)에서 모델 정확도를 확인하세요.

최적화된 모델 결과를 원본과 비교하려면 검증 데이터 세트의 모델 정확도 측면에서 측정 가능한 몇 가지 결과를 아는 것이 좋습니다.


### 검증 데이터 세트 다운로드 [$\Uparrow$](#목차:)

YOLOv8은 COCO 데이터세트에 대해 사전 훈련되었으므로 모델 정확도를 평가하려면 이를 다운로드해야 합니다. YOLOv8 저장소에 제공된 지침에 따라 원래 모델 평가 기능과 함께 사용하려면 모델 작성자가 사용하는 형식으로 주석을 다운로드해야 합니다.

>**참고**: 초기 데이터 세트 다운로드를 완료하는 데 몇 분 정도 걸릴 수 있습니다. 다운로드 속도는 인터넷 연결 품질에 따라 달라집니다.

In [ ]:
from zipfile import ZipFile

DATA_URL = "http://images.cocodataset.org/zips/val2017.zip"
LABELS_URL = "https://github.com/ultralytics/yolov5/releases/download/v1.0/coco2017labels-segments.zip"
CFG_URL = "https://raw.githubusercontent.com/ultralytics/ultralytics/8ebe94d1e928687feaa1fee6d5668987df5e43be/ultralytics/datasets/coco.yaml"

OUT_DIR = Path('./datasets')

DATA_PATH = OUT_DIR / "val2017.zip"
LABELS_PATH = OUT_DIR / "coco2017labels-segments.zip"
CFG_PATH = OUT_DIR / "coco.yaml"

download_file(DATA_URL, DATA_PATH.name, DATA_PATH.parent)
download_file(LABELS_URL, LABELS_PATH.name, LABELS_PATH.parent)
download_file(CFG_URL, CFG_PATH.name, CFG_PATH.parent)

if not (OUT_DIR / "coco/labels").exists():
    with ZipFile(LABELS_PATH , "r") as zip_ref:
        zip_ref.extractall(OUT_DIR)
    with ZipFile(DATA_PATH , "r") as zip_ref:
        zip_ref.extractall(OUT_DIR / 'coco/images')

### 유효성 검사 함수 정의 [$\Uparrow$](#목차:)


In [ ]:
from tqdm.notebook import tqdm
from ultralytics.yolo.utils.metrics import ConfusionMatrix


def test(model:ov.Model, core:ov.Core, data_loader:torch.utils.data.DataLoader, validator, num_samples:int = None):
    """
    OpenVINO YOLOv8 model accuracy validation function. Runs model validation on dataset and returns metrics
    Parameters:
        model (Model): OpenVINO model
        data_loader (torch.utils.data.DataLoader): dataset loader
        validator: instance of validator class
        num_samples (int, *optional*, None): validate model only on specified number samples, if provided
    Returns:
        stats: (Dict[str, float]) - dictionary with aggregated accuracy metrics statistics, key is metric name, value is metric value
    """
    validator.seen = 0
    validator.jdict = []
    validator.stats = []
    validator.batch_i = 1
    validator.confusion_matrix = ConfusionMatrix(nc=validator.nc)
    model.reshape({0: [1, 3, -1, -1]})
    compiled_model = core.compile_model(model)
    for batch_i, batch in enumerate(tqdm(data_loader, total=num_samples)):
        if num_samples is not None and batch_i == num_samples:
            break
        batch = validator.preprocess(batch)
        results = compiled_model(batch["img"])
        preds = torch.from_numpy(results[compiled_model.output(0)])
        preds = validator.postprocess(preds)
        validator.update_metrics(preds, batch)
    stats = validator.get_stats()
    return stats


def print_stats(stats:np.ndarray, total_images:int, total_objects:int):
    """
    Helper function for printing accuracy statistic
    Parameters:
        stats: (Dict[str, float]) - dictionary with aggregated accuracy metrics statistics, key is metric name, value is metric value
        total_images (int) -  number of evaluated images
        total objects (int)
    Returns:
        None
    """
    print("Boxes:")
    mp, mr, map50, mean_ap = stats['metrics/precision(B)'], stats['metrics/recall(B)'], stats['metrics/mAP50(B)'], stats['metrics/mAP50-95(B)']
    # Print results
    s = ('%20s' + '%12s' * 6) % ('Class', 'Images', 'Labels', 'Precision', 'Recall', 'mAP@.5', 'mAP@.5:.95')
    print(s)
    pf = '%20s' + '%12i' * 2 + '%12.3g' * 4  # print format
    print(pf % ('all', total_images, total_objects, mp, mr, map50, mean_ap))
    if 'metrics/precision(M)' in stats:
        s_mp, s_mr, s_map50, s_mean_ap = stats['metrics/precision(M)'], stats['metrics/recall(M)'], stats['metrics/mAP50(M)'], stats['metrics/mAP50-95(M)']
        # Print results
        s = ('%20s' + '%12s' * 6) % ('Class', 'Images', 'Labels', 'Precision', 'Recall', 'mAP@.5', 'mAP@.5:.95')
        print(s)
        pf = '%20s' + '%12i' * 2 + '%12.3g' * 4  # print format
        print(pf % ('all', total_images, total_objects, s_mp, s_mr, s_map50, s_mean_ap))

### 유효성 검사 도우미 구성 및 DataLoader 생성 [$\Uparrow$](#Table-of-content:)

원본 모델 저장소는 정확성 검증 파이프라인을 나타내는 'Validator' 래퍼를 사용합니다. 데이터로더와 평가 지표를 생성하고 데이터로더에서 생성된 각 데이터 배치에 대한 지표를 업데이트합니다. 그 외에도 데이터 전처리와 결과 후처리를 담당합니다. 클래스 초기화를 위해서는 구성을 제공해야 합니다. 기본 설정을 사용하지만 사용자 정의 데이터를 테스트하기 위해 재정의하는 일부 매개변수로 대체할 수 있습니다. 모델은 유효성 검사기 클래스 인스턴스를 생성하는 'ValidatorClass' 메서드를 연결했습니다.

In [ ]:
from ultralytics.yolo.utils import DEFAULT_CFG
from ultralytics.yolo.cfg import get_cfg
from ultralytics.yolo.data.utils import check_det_dataset

args = get_cfg(cfg=DEFAULT_CFG)
args.data = str(CFG_PATH)

In [ ]:
det_validator = det_model.ValidatorClass(args=args)

In [ ]:
det_validator.data = check_det_dataset(args.data)
det_data_loader = det_validator.get_dataloader("datasets/coco", 1)

In [ ]:
det_validator.is_coco = True
det_validator.class_map = ops.coco80_to_coco91_class()
det_validator.names = det_model.model.names
det_validator.metrics.names = det_validator.names
det_validator.nc = det_model.model.model[-1].nc

정의 테스트 기능 및 유효성 검사기 생성 후 정확도 측정항목을 얻을 준비가 되었습니다.
>**참고**: 모델 평가는 시간이 많이 걸리는 프로세스이며 하드웨어에 따라 몇 분 정도 걸릴 수 있습니다. 계산 시간을 줄이기 위해 'num_samples' 매개변수를 평가 하위 집합 크기로 정의하지만 이 경우 검증 하위 집합 차이로 인해 모델 작성자가 원래 보고한 정확도와 비교할 수 없을 수 있습니다.
*전체 데이터 세트 `NUM_TEST_SAMPLES = None`에서 모델을 검증합니다.*

In [ ]:
NUM_TEST_SAMPLES = 300

In [ ]:
fp_det_stats = test(det_ov_model, core, det_data_loader, det_validator, num_samples=NUM_TEST_SAMPLES)

In [ ]:
print_stats(fp_det_stats, det_validator.seen, det_validator.nt_per_class.sum())

`print_stats`는 다음과 같은 정확도 지표 목록을 보고합니다.

* '정밀도'는 해당 객체만 식별하는 모델의 정확성 정도입니다.
* 'Recall'은 모든 실제 객체를 감지하는 모델의 능력을 측정합니다.
* `mAP@t` - 데이터 세트의 모든 클래스에 대해 집계된 Precision-Recall 곡선 아래 영역으로 표시되는 평균 정밀도를 의미합니다. 여기서 `t`는 IOU(Intersection Over Union) 임계값, 실제와 예측 사이의 중첩 정도입니다. 사물. 따라서 'mAP@.5'는 평균 정밀도가 0.5 IOU 임계값에서 계산됨을 나타내고, 'mAP@.5:.95'는 0.05 단계를 통해 0.5에서 0.95까지의 IOU 임계값 범위에서 계산됩니다.

## NNCF 사후 훈련 양자화 API [$\Uparrow$](#Table-of-content:)를 사용하여 모델 최적화

[NNCF](https://github.com/openvinotoolkit/nncf)는 OpenVINO에서 정확도 저하를 최소화하면서 신경망 추론 최적화를 위한 고급 알고리즘 제품군을 제공합니다.
YOLOv8을 최적화하기 위해 사후 훈련 모드(미세 조정 파이프라인 없이)에서 8비트 양자화를 사용합니다.

최적화 프로세스에는 다음 단계가 포함됩니다.

1. 양자화를 위한 데이터 세트를 생성합니다.
2. 최적화된 모델을 얻으려면 'nncf.Quantize'를 실행하세요.
3. 'openvino.runtime.serialize' 함수를 사용하여 OpenVINO IR 모델을 직렬화합니다.

양자화 정확도 테스트에 검증 데이터로더를 재사용합니다.
이를 위해서는 `nncf.Dataset` 객체로 래핑하고 입력 텐서만 가져오기 위한 변환 함수를 정의해야 합니다.

In [ ]:
import nncf  # noqa: F811
from typing import Dict


def transform_fn(data_item:Dict):
    """
    Quantization transform function. Extracts and preprocess input data from dataloader item for quantization.
    Parameters:
       data_item: Dict with data item produced by DataLoader during iteration
    Returns:
        input_tensor: Input data for quantization
    """
    input_tensor = det_validator.preprocess(data_item)['img'].numpy()
    return input_tensor


quantization_dataset = nncf.Dataset(det_data_loader, transform_fn)

'nncf.Quantize' 함수는 모델 양자화를 위한 인터페이스를 제공합니다. OpenVINO 모델 및 양자화 데이터 세트의 인스턴스가 필요합니다.
선택적으로 구성 양자화 프로세스를 위한 일부 추가 매개변수(양자화를 위한 샘플 수, 사전 설정, 무시된 범위 등)를 제공할 수 있습니다. YOLOv8 모델에는 활성화의 비대칭 양자화가 필요한 비ReLU 활성화 함수가 포함되어 있습니다. 더 나은 결과를 얻기 위해 '혼합' 양자화 사전 설정을 사용합니다. 이는 가중치의 대칭 양자화와 활성화의 비대칭 양자화를 제공합니다. 보다 정확한 결과를 얻으려면 `ignored_scope` 매개변수를 사용하여 후처리 하위 그래프의 작업을 부동 소수점 정밀도로 유지해야 합니다.

>**참고**: 모델 학습 후 양자화는 시간이 많이 걸리는 프로세스입니다. 인내심을 가지세요. 하드웨어에 따라 몇 분 정도 걸릴 수 있습니다.

In [ ]:
ignored_scope = nncf.IgnoredScope(
    types=["Multiply", "Subtract", "Sigmoid"],  # ignore operations
    names=[
        "/model.22/dfl/conv/Conv",           # in the post-processing subgraph
        "/model.22/Add",
        "/model.22/Add_1",
        "/model.22/Add_2",
        "/model.22/Add_3",
        "/model.22/Add_4",
        "/model.22/Add_5",
        "/model.22/Add_6",
        "/model.22/Add_7",
        "/model.22/Add_8",
        "/model.22/Add_9",
        "/model.22/Add_10"
    ]
)


# Detection model
quantized_det_model = nncf.quantize(
    det_ov_model,
    quantization_dataset,
    preset=nncf.QuantizationPreset.MIXED,
    ignored_scope=ignored_scope
)

In [ ]:
from openvino.runtime import serialize
int8_model_det_path = models_dir / f'{DET_MODEL_NAME}_openvino_int8_model/{DET_MODEL_NAME}.xml'
print(f"Quantized detection model will be saved to {int8_model_det_path}")
serialize(quantized_det_model, str(int8_model_det_path))

### 양자화 모델 추론 검증 [$\Uparrow$](#목차:)

`nncf.Quantize`는 예측을 위해 장치에 로드하는 데 적합한 OpenVINO 모델 클래스 인스턴스를 반환합니다. `INT8` 모델 입력 데이터 및 출력 결과 형식은 부동 소수점 모델 표현과 차이가 없습니다. 따라서 이미지에서 'INT8' 모델 결과를 얻기 위해 위에서 정의한 것과 동일한 'Detect' 함수를 재사용할 수 있습니다.

In [ ]:
device

In [ ]:
if device.value != "CPU":
    quantized_det_model.reshape({0: [1, 3, 640, 640]})
quantized_det_compiled_model = core.compile_model(quantized_det_model, device.value)
input_image = np.array(Image.open(IMAGE_PATH))
detections = detect(input_image, quantized_det_compiled_model)[0]
image_with_boxes = draw_results(detections, input_image, label_map)

Image.fromarray(image_with_boxes)

## 원본 모델과 양자화 모델 비교 [$\Uparrow$](#목차:)

### 성능 개체 감지 모델 비교 [$\Uparrow$](#목차:)

마지막으로 OpenVINO [벤치마크 도구](https://docs.openvino.ai/2023.0/openvino_inference_engine_tools_benchmark_tool_README.html)를 사용하여 'FP32' 및 'INT8' 모델의 추론 성능을 측정합니다.

> **참고**: 보다 정확한 성능을 위해서는 다른 애플리케이션을 닫은 후 터미널/명령 프롬프트에서 `benchmark_app`을 실행하는 것이 좋습니다. `benchmark_app -m <model_path> -d CPU -shape "<input_shape>"`를 실행하여 특정 입력 데이터 형태에 대한 CPU의 비동기 추론을 1분 동안 벤치마킹합니다. GPU를 벤치마킹하려면 'CPU'를 'GPU'로 변경하세요. 모든 명령줄 옵션의 개요를 보려면 `benchmark_app --help`를 실행하세요.

In [ ]:
device

In [ ]:
# Inference FP32 model (OpenVINO IR)
!benchmark_app -m $det_model_path -d $device.value -api async -shape "[1,3,640,640]"

In [ ]:
# Inference INT8 model (OpenVINO IR)
!benchmark_app -m $int8_model_det_path -d $device.value -api async -shape "[1,3,640,640]" -t 15

### 양자화된 모델 정확도 검증 [$\Uparrow$](#목차:)

보시다시피 단일 이미지 테스트에서는 `INT8`과 부동소수점 모델 결과 사이에 큰 차이가 없습니다. 양자화가 모델 예측 정밀도에 어떻게 영향을 미치는지 이해하기 위해 데이터 세트에서 모델 정확도를 비교할 수 있습니다.

In [ ]:
int8_det_stats = test(quantized_det_model, core, det_data_loader, det_validator, num_samples=NUM_TEST_SAMPLES)

In [ ]:
print("FP32 model accuracy")
print_stats(fp_det_stats, det_validator.seen, det_validator.nt_per_class.sum())

print("INT8 model accuracy")
print_stats(int8_det_stats, det_validator.seen, det_validator.nt_per_class.sum())

엄청난! 정확도가 변경된 것처럼 보이지만 크게 변경되지 않았으며 통과 기준을 충족합니다.

## 다음 단계 [$\Uparrow$](#목차:)
이 섹션에는 OpenVINO를 사용하여 애플리케이션 성능을 추가로 향상시키는 방법에 대한 제안 사항이 포함되어 있습니다.

### 비동기 추론 파이프라인 [$\Uparrow$](#목차:)
Async API의 주요 장점은 장치가 추론으로 바쁜 경우 애플리케이션이 현재 추론이 먼저 완료될 때까지 기다리지 않고 다른 작업(예: 입력 채우기 또는 다른 요청 예약)을 병렬로 수행할 수 있다는 것입니다. openvino를 사용하여 비동기 추론을 수행하는 방법을 이해하려면 [Async API 튜토리얼](../115-async-api/115-async-api.ipynb)을 참조하세요.

### 모델 [$\Uparrow$](#목차:)에 대한 통합 전처리

전처리 API를 사용하면 전처리를 모델의 일부로 만들어 애플리케이션 코드와 추가 이미지 처리 라이브러리에 대한 종속성을 줄일 수 있습니다.
전처리 API의 주요 장점은 전처리 단계가 실행 그래프에 통합되어 애플리케이션의 일부로 CPU에서 항상 실행되는 대신 선택한 장치(CPU/GPU 등)에서 수행된다는 것입니다. 이렇게 하면 선택한 장치 활용도가 향상됩니다.

자세한 내용은 [전처리 API](https://docs.openvino.ai/2023.0/openvino_docs_OV_Runtime_UG_Preprocessing_Overview.html) 개요를 참조하세요.

예를 들어 'image_to_tensor' 함수에 정의된 입력 데이터 레이아웃 변환과 정규화를 통합할 수 있습니다.

통합 프로세스는 다음 단계로 구성됩니다.
1. PrePostProcessing 객체를 초기화합니다.
2. 입력 데이터 형식을 정의합니다.
3. 전처리 단계를 설명하세요.
4. 모델에 단계 통합.

#### PrePostProcessing API 초기화 [$\Uparrow$](#목차:)

'openvino.preprocess.PrePostProcessor' 클래스를 사용하면 모델의 전처리 및 후처리 단계를 지정할 수 있습니다.

In [ ]:
from openvino.preprocess import PrePostProcessor

ppp = PrePostProcessor(quantized_det_model)

#### 입력 데이터 형식 정의 [$\Uparrow$](#목차:)
모델/전처리기의 특정 입력을 처리하기 위해 `input(input_id)` 메서드. 여기서 `input_id`는 `model.inputs` 입력에 대한 위치 인덱스 또는 입력 텐서 이름입니다. 모델에 단일 입력 `input_id가 있는 경우 `는 생략 가능합니다.
디스크에서 이미지를 읽은 후 '[0, 255]' 범위의 U8 픽셀을 포함하고 'NHWC' 레이아웃에 저장됩니다. 전처리 변환을 수행하려면 이를 텐서 설명에 제공해야 합니다.

In [ ]:
ppp.input(0).tensor().set_shape([1, 640, 640, 3]).set_element_type(ov.Type.u8).set_layout(ov.Layout('NHWC'))
pass

레이아웃 변환을 수행하려면 모델에서 예상하는 레이아웃에 대한 정보도 제공해야 합니다.

#### 전처리 단계 설명 [$\Uparrow$](#목차:)

전처리 기능에는 다음 단계가 포함됩니다.
* 데이터 유형을 'U8'에서 'FP32'로 변환합니다.
* 데이터 레이아웃을 'NHWC'에서 'NCHW' 형식으로 변환합니다.
* 배율 255로 나누어 각 픽셀을 정규화합니다.

`ppp.input(input_id).preprocess()`는 일련의 전처리 단계를 정의하는 데 사용됩니다.

In [ ]:
ppp.input(0).preprocess().convert_element_type(ov.Type.f32).convert_layout(ov.Layout('NCHW')).scale([255., 255., 255.])

print(ppp)

#### 모델에 단계 통합 [$\Uparrow$](#목차:)

전처리 단계가 완료되면 최종적으로 모델을 구축할 수 있습니다. 또한 `openvino.runtime.serialize`를 사용하여 완성된 모델을 OpenVINO IR에 저장할 수 있습니다.

In [ ]:
quantized_model_with_preprocess = ppp.build()
serialize(quantized_model_with_preprocess, str(int8_model_det_path.with_name(f"{DET_MODEL_NAME}_with_preprocess.xml")))

전처리가 통합된 모델을 기기에 로드할 준비가 되었습니다. 이제 감지 기능에서 다음 전처리 단계를 건너뛸 수 있습니다.

In [ ]:
def detect_without_preprocess(image:np.ndarray, model:ov.Model):
    """
    OpenVINO YOLOv8 model with integrated preprocessing inference function. Preprocess image, runs model inference and postprocess results using NMS.
    Parameters:
        image (np.ndarray): input image.
        model (Model): OpenVINO compiled model.
    Returns:
        detections (np.ndarray): detected boxes in format [x1, y1, x2, y2, score, label]
    """
    output_layer = model.output(0)
    img = letterbox(image)[0]
    input_tensor = np.expand_dims(img, 0)
    input_hw = img.shape[:2]
    result = model(input_tensor)[output_layer]
    detections = postprocess(result, input_hw, image)
    return detections


compiled_model = core.compile_model(quantized_model_with_preprocess, device.value)
input_image = np.array(Image.open(IMAGE_PATH))
detections = detect_without_preprocess(input_image, compiled_model)[0]
image_with_boxes = draw_results(detections, input_image, label_map)

Image.fromarray(image_with_boxes)

## 라이브 데모 [$\Uparrow$](#목차:)

다음 코드는 비디오에서 모델 추론을 실행합니다.

In [ ]:
import collections
import time
from IPython import display


# Main processing function to run object detection.
def run_object_detection(source=0, flip=False, use_popup=False, skip_first_frames=0, model=det_model, device=device.value):
    player = None
    if device != "CPU":
        model.reshape({0: [1, 3, 640, 640]})
    compiled_model = core.compile_model(model, device)
    try:
        # Create a video player to play with target fps.
        player = VideoPlayer(
            source=source, flip=flip, fps=30, skip_first_frames=skip_first_frames
        )
        # Start capturing.
        player.start()
        if use_popup:
            title = "Press ESC to Exit"
            cv2.namedWindow(
                winname=title, flags=cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE
            )

        processing_times = collections.deque()
        while True:
            # Grab the frame.
            frame = player.next()
            if frame is None:
                print("Source ended")
                break
            # If the frame is larger than full HD, reduce size to improve the performance.
            scale = 1280 / max(frame.shape)
            if scale < 1:
                frame = cv2.resize(
                    src=frame,
                    dsize=None,
                    fx=scale,
                    fy=scale,
                    interpolation=cv2.INTER_AREA,
                )
            # Get the results.
            input_image = np.array(frame)

            start_time = time.time()
            # model expects RGB image, while video capturing in BGR
            detections = detect(input_image[:, :, ::-1], compiled_model)[0]
            stop_time = time.time()

            image_with_boxes = draw_results(detections, input_image, label_map)
            frame = image_with_boxes

            processing_times.append(stop_time - start_time)
            # Use processing times from last 200 frames.
            if len(processing_times) > 200:
                processing_times.popleft()

            _, f_width = frame.shape[:2]
            # Mean processing time [ms].
            processing_time = np.mean(processing_times) * 1000
            fps = 1000 / processing_time
            cv2.putText(
                img=frame,
                text=f"Inference time: {processing_time:.1f}ms ({fps:.1f} FPS)",
                org=(20, 40),
                fontFace=cv2.FONT_HERSHEY_COMPLEX,
                fontScale=f_width / 1000,
                color=(0, 0, 255),
                thickness=1,
                lineType=cv2.LINE_AA,
            )
            # Use this workaround if there is flickering.
            if use_popup:
                cv2.imshow(winname=title, mat=frame)
                key = cv2.waitKey(1)
                # escape = 27
                if key == 27:
                    break
            else:
                # Encode numpy array to jpg.
                _, encoded_img = cv2.imencode(
                    ext=".jpg", img=frame, params=[cv2.IMWRITE_JPEG_QUALITY, 100]
                )
                # Create an IPython image.
                i = display.Image(data=encoded_img)
                # Display the image in this notebook.
                display.clear_output(wait=True)
                display.display(i)
    # ctrl-c
    except KeyboardInterrupt:
        print("Interrupted")
    # any different error
    except RuntimeError as e:
        print(e)
    finally:
        if player is not None:
            # Stop capturing.
            player.stop()
        if use_popup:
            cv2.destroyAllWindows()

### 실시간 개체 감지 실행 [$\Uparrow$](#목차:)

웹캠을 비디오 입력으로 사용하십시오. 기본적으로 기본 웹캠은 `source=0`으로 설정됩니다. 웹캠이 여러 개인 경우 각 웹캠에는 0부터 시작하는 연속 번호가 할당됩니다. 전면 카메라를 사용할 때는 `flip=True`를 설정하세요. 일부 웹 브라우저, 특히 Mozilla Firefox에서는 깜박임이 발생할 수 있습니다. 깜박임이 발생하면 `use_popup=True`를 설정하세요.

>**참고**: 이 노트북을 웹캠과 함께 사용하려면 웹캠이 있는 컴퓨터에서 노트북을 실행해야 합니다. 원격 서버(예: Binder 또는 Google Colab 서비스)에서 노트북을 실행하면 웹캠이 작동하지 않습니다. 기본적으로 아래쪽 셀은 비디오 파일에 대한 모델 추론을 실행합니다. 웹캠에서 실시간 추론을 시도하려면 'WEBCAM_INFERENCE = True'로 설정하세요.

객체 감지를 실행합니다.

In [ ]:
WEBCAM_INFERENCE = False

if WEBCAM_INFERENCE:
    VIDEO_SOURCE = 0  # Webcam
else:
    VIDEO_SOURCE = 'https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/video/people.mp4'

In [ ]:
device

In [ ]:
run_object_detection(source=VIDEO_SOURCE, flip=True, use_popup=False, model=det_ov_model, device=device.value)